In [4]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from models.url_ml_catboost_classifier import SingletonURLMLCatBoostClassifier
from models.url_ml_desicion_tree_classifier import SingletonURLMLDecisionTreeClassifier
from models.url_ml_gradient_boosting_classifier import (
    SingletonURLMLGradientBoostingClassifier,
)
from models.url_ml_knn_classifier import SingletonURLMLKNNClassifier
from models.url_ml_logistic_regression_classifier import (
    SingletonURLMLLogisticRegressionClassifier,
)
from models.url_ml_mlp_classifier import SingletonURLMLMLPClassifier
from models.url_ml_naive_bayes_classifier import SingletonURLMLNaiveBayesClassifier
from models.url_ml_random_forest import SingletonURLMLRandomForestClassifier
from models.url_ml_svm_classifier import SingletonURLMLSVMClassifier
from utils.url_utils import load_data_from_csv

In [6]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score


def evaluate_classifiers(classifiers, X_train, Y_train, X_test, Y_test, is_training):
    accuracies = []
    f1_scores = []
    recalls = []
    precisions = []
    
    for classifier in classifiers:
        if is_training:
            classifier.train(X_train.copy(), Y_train.copy())
            classifier.save()
        else:
            classifier.load()

        print(f"Evaluating {classifier.model_name}...")
        Y_pred = classifier.predict(X_test.copy())
        
        accuracy = accuracy_score(Y_test.copy(), Y_pred)
        f1 = f1_score(Y_test.copy(), Y_pred, average='weighted')
        recall = recall_score(Y_test.copy(), Y_pred, average='weighted')
        precision = precision_score(Y_test.copy(), Y_pred, average='weighted')
        
        print(f"Accuracy: {accuracy}")
        print(f"F1 Score: {f1}")
        print(f"Recall: {recall}")
        print(f"Precision: {precision}")
        
        accuracies.append(accuracy * 100)
        f1_scores.append(f1 * 100)
        recalls.append(recall * 100)
        precisions.append(precision * 100)
    
    return accuracies, f1_scores, recalls, precisions

def plot_metrics(model_names, accuracies, f1_scores, recalls, precisions):
    metrics_df = pd.DataFrame({
        'Model': model_names,
        'Accuracy (%)': accuracies,
        'F1 Score (%)': f1_scores,
        'Recall (%)': recalls,
        'Precision (%)': precisions
    })
    display(metrics_df)

def plot_accuracies(model_names, accuracies):
    plt.figure(figsize=(12, 6))
    plt.bar(model_names, accuracies)
    plt.xlabel("Model")
    plt.ylabel("Accuracy (%)")
    plt.title("Model Comparison")
    for i, accuracy in enumerate(accuracies):
        plt.text(i, accuracy + 1, f"{accuracy:.2f}%", ha="center", va="bottom")
    plt.show()

In [7]:
URL_CSV_PATH = "./data/url-data.csv"
MODEL_DIR = "./trained_models"
IS_TRAINING = False

classifiers = [
    SingletonURLMLSVMClassifier(MODEL_DIR),
    SingletonURLMLRandomForestClassifier(MODEL_DIR),
    SingletonURLMLLogisticRegressionClassifier(MODEL_DIR),
    SingletonURLMLKNNClassifier(MODEL_DIR),
    SingletonURLMLGradientBoostingClassifier(MODEL_DIR),
    SingletonURLMLCatBoostClassifier(MODEL_DIR),
    SingletonURLMLDecisionTreeClassifier(MODEL_DIR),
    SingletonURLMLMLPClassifier(MODEL_DIR),
    SingletonURLMLNaiveBayesClassifier(MODEL_DIR),
]
model_names = [classifier.model_name for classifier in classifiers]

X, Y = load_data_from_csv(URL_CSV_PATH)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)
accuracies = evaluate_classifiers(
    classifiers, X_train, Y_train, X_test, Y_test, IS_TRAINING
)

plot_metrics(model_names, *accuracies)

# plot_accuracies(model_names, accuracies)

Evaluating svm...
Accuracy: 0.9642695612844867
F1 Score: 0.9642074624721793
Recall: 0.9642695612844867
Precision: 0.9644836492759153
Evaluating random-forest...
Accuracy: 0.9665309814563546
F1 Score: 0.9665161410202683
Recall: 0.9665309814563546
Precision: 0.9665269789427762
Evaluating logistic-regression...
Accuracy: 0.9335142469470827
F1 Score: 0.9333898392567606
Recall: 0.9335142469470827
Precision: 0.933639375574251
Evaluating knn...
Accuracy: 0.9561284486657621
F1 Score: 0.9561212943256517
Recall: 0.9561284486657621
Precision: 0.9561177071574319
Evaluating gradient-boosting...
Accuracy: 0.9742198100407056
F1 Score: 0.9741750045685343
Recall: 0.9742198100407056
Precision: 0.9744721068982471
Evaluating catboost...
Accuracy: 0.9724106739032112
F1 Score: 0.9723837745188059
Recall: 0.9724106739032112
Precision: 0.9724737960846602
Evaluating decision-tree...
Accuracy: 0.9597467209407508
F1 Score: 0.959744550016864
Recall: 0.9597467209407508
Precision: 0.9597427768258082
Evaluating mlp..

,Model,Accuracy (%),F1 Score (%),Recall (%),Precision (%)
0,svm,96.426956,96.420746,96.426956,96.448365
1,random-forest,96.653098,96.651614,96.653098,96.652698
2,logistic-regression,93.351425,93.338984,93.351425,93.363938
3,knn,95.612845,95.612129,95.612845,95.611771
4,gradient-boosting,97.421981,97.417500,97.421981,97.447211
5,catboost,97.241067,97.238377,97.241067,97.247380
6,decision-tree,95.974672,95.974455,95.974672,95.974278
7,mlp,97.241067,97.236993,97.241067,97.258254
8,naive-bayes,60.470375,55.816592,60.470375,78.842023
